In [1]:
%load_ext lab_black
%cd ..

/data3/SIG/log-analytics


In [2]:
import argparse
import math
import multiprocessing
import sys
from datetime import datetime
from pathlib import Path
from pprint import pformat

import numpy as np
import pandas as pd
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch_optimizer
import yaml
from easydict import EasyDict
from pytorch_transformers import (
    BertForSequenceClassification,
    BertTokenizer,
    DistilBertForSequenceClassification,
    DistilBertTokenizer,
    RobertaForSequenceClassification,
    RobertaTokenizer,
)
from sklearn.model_selection import StratifiedKFold
from torch.optim import Adam, AdamW
from torch.optim.lr_scheduler import ReduceLROnPlateau
from torch.utils.data import DataLoader, Dataset
from tqdm import tqdm
from transformers import (
    AlbertForSequenceClassification,
    AlbertTokenizer,
    DebertaForSequenceClassification,
    DebertaTokenizer,
    SqueezeBertTokenizer,
    SqueezeBertForSequenceClassification,
    XLNetTokenizer,
    XLNetForSequenceClassification,
)

from datasets import load_test_data, load_train_data, MyDataset, load_train_total_data
from utils import SAM, AverageMeter, CustomLogger, FocalLoss, seed_everything

from main import MyTrainer
from collections import defaultdict
import matplotlib.pyplot as plt
import random

In [3]:
import os

os.environ["CUDA_VISIBLE_DEVICES"] = "3"
torch.set_grad_enabled(False)

In [4]:
with open("config/distilbert-base-uncased-ver2.yaml", "r") as f:
    C = EasyDict(yaml.load(f, yaml.FullLoader))
    C.result_dir = Path(C.result_dir)
    C.dataset.dir = Path(C.dataset.dir)
    seed_everything(C.seed, deterministic=False)

In [5]:
C

{'model': {'name': 'distilbert-base-uncased'},
 'comment': None,
 'result_dir': PosixPath('results/distilbert-base-uncased'),
 'debug': False,
 'seed': 20210425,
 'train': {'SAM': False,
  'folds': [1],
  'checkpoints': [None],
  'loss': {'name': 'focal', 'gamma': 2},
  'optimizer': {'name': 'AdamW'},
  'finetune': {'do': True, 'step1_epochs': 2, 'step2_epochs': 4},
  'max_epochs': 10,
  'lr': 1e-05,
  'scheduler': {'name': 'ReduceLROnPlateau',
   'params': {'factor': 0.5, 'patience': 3, 'verbose': True}}},
 'dataset': {'dir': PosixPath('data/ori'),
  'ver': 2,
  'batch_size': 30,
  'num_workers': 8}}

## deck결과 연구

In [26]:
decks = [
    np.load("results/distilbert-base-uncased/distilbert-base-uncased-focal-AdamW-lr1e-05-dsver2_1-deck3.npz"),
    np.load("results/distilbert-base-uncased/distilbert-base-uncased-focal-AdamW-lr1e-05-dsver2_2-deck3.npz"),
    np.load("results/distilbert-base-uncased/distilbert-base-uncased-focal-AdamW-lr1e-05-dsver2_3-deck3.npz"),
]

In [27]:
for i in range(len(decks)):
    decks[i] = {k: torch.from_numpy(v) for k, v in decks[i].items()}

### deck 합치기

In [28]:
decks[0].keys()

dict_keys(['dists', 'indices', 'fcfeats', 'tlevels'])

In [29]:
deck = {
    "dists": torch.cat([decks[i]["dists"] for i in range(len(decks))], dim=1),
    "tlevels": torch.cat([decks[i]["tlevels"] for i in range(len(decks))], dim=1),
    "indices": torch.cat([decks[i]["indices"] for i in range(len(decks))], dim=1),
}

In [30]:
deck["dists"][10].shape

torch.Size([24])

In [31]:
out_deck = {"dists": [], "tlevels": [], "indices": []}
with tqdm(total=1418916, ncols=100, file=sys.stdout) as t:
    for i in range(1418916):
        dist, j = deck["dists"][i].topk(8, dim=0, largest=False)
        tlevel = deck["tlevels"][i, j]
        index = deck["indices"][i, j]
        out_deck["dists"].append(dist)
        out_deck["tlevels"].append(tlevel)
        out_deck["indices"].append(index)
        t.update()

100%|██████████████████████████████████████████████████| 1418916/1418916 [00:39<00:00, 35905.41it/s]


In [32]:
deck = out_deck

In [33]:
deck["dists"] = torch.stack(deck["dists"])
deck["tlevels"] = torch.stack(deck["tlevels"])
deck["indices"] = torch.stack(deck["indices"])

In [34]:
np.savez_compressed(
    "results/distilbert-base-uncased/distilbert-base-uncased-focal-AdamW-lr1e-05-dsver2-total_deck.npz",
    dists=deck["dists"].numpy(),
    tlevels=deck["tlevels"].numpy(),
    indices=deck["indices"].numpy(),
)

## submission 파일 만들기

In [36]:
# ver4
def politic_draw(dists, indices, tlevels):
    dd = dists[-1] - dists[0]
    dist = dists[0]
    same = (tlevels == tlevels[0]).sum() == 8

    # policy1: dist와 관계 없이 모든 tlevels가 3 또는 5이면 그 값을 출력
    if same and tlevels[0] in [3, 5]:
        return tlevels[0].item()

    # policy2: dist와 관계 없이 모든 tlevels 중 앞의 4개가 2 또는 4 또는 6이면 그 값을 출력
    if tlevels[0] in [2, 4, 6] and (tlevels[1:4] == tlevels[0]).sum() == 3:
        return tlevels[0].item()

    # policy: dist가 0.4보다 크면 level 7
    if dist > 0.4:
        return 7

    # 나머지
    return tlevels[0].item()

In [37]:
i = 100
politic_draw(deck["dists"][i], deck["indices"][i], deck["tlevels"][i])

0

In [38]:
N = len(deck["dists"])
outdic = {"id": list(range(1000000, 2418915 + 1)), "level": []}
with tqdm(total=N, ncols=100, file=sys.stdout) as t:
    for i in range(N):
        v = politic_draw(deck["dists"][i], deck["indices"][i], deck["tlevels"][i])
        outdic["level"].append(v)
        t.update()

100%|███████████████████████████████████████████████████| 1418916/1418916 [02:22<00:00, 9975.51it/s]


In [39]:
outdf = pd.DataFrame(outdic)

In [40]:
outdf.to_csv("results/distilbert-base-uncased/distilbert-base-uncased-focal-AdamW-lr1e-05-dsver2-total-ver4.csv", index=False)